In [23]:
# pip install --user albumentations

## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from torch.utils.data import DataLoader, RandomSampler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm import tqdm
import timm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Settting

In [24]:
CFG = {}
CFG['SEED'] = 42
CFG['N_SPLIT'] = 5
CFG['LABEL_SMOOTHING'] = 0.05
CFG['OPTIMIZER'] = 'AdamW'
CFG['MODEL_NAME'] = "timm/deit3_large_patch16_224.fb_in22k_ft_in1k"    ## 304MB
CFG['IMG_SIZE'] = 224
CFG['BATCH_SIZE'] = 8 ## 48//16G, 4//8G memory..
CFG['LEARNING_RATE']= 3e-4
CFG['EPOCHS']= 5

## Fixed RandomSeed

In [25]:
def seed_everything(seed):
    random.seed(seed) ##random module의 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed) #해시 함수의 랜덤성 제어, 자료구조 실행할 때 동일한 순서 고정
    np.random.seed(seed) #numpy 랜덤 숫자 일정
    torch.manual_seed(seed) # torch라이브러리에서 cpu 텐서 생성 랜덤 시드 고정
    torch.cuda.manual_seed(seed) # cuda의 gpu텐서에 대한 시드 고정
    torch.backends.cudnn.deterministic = True # 백엔드가 결정적 알고리즘만 사용하도록 고정
    torch.backends.cudnn.benchmark = True # CuDNN이 여러 내부 휴리스틱을 사용하여 가장 빠른 알고리즘 동적으로 찾도록 설정

## Train & Validation Split

In [15]:
# CSV 파일 로드 및 train-test split
df = pd.read_csv('/home/idp/lab/minjun/dl/project/train.csv')
train_df, val_df, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])
test_df = pd.read_csv('/home/idp/lab/minjun/dl/project/test.csv')

# 경로 수정
base_path = '/home/idp/lab/minjun/dl/project/'

# train, val, test 데이터프레임에서 경로 수정 (앞에 base_path 추가 후 ./ 제거)
train_df['upscale_img_path'] = base_path + train_df['upscale_img_path'].str.lstrip('./')
val_df['upscale_img_path'] = base_path + val_df['upscale_img_path'].str.lstrip('./')
test_df['img_path'] = base_path + test_df['img_path'].str.lstrip('./')

# 경로 수정 확인
print("Train paths:", train_df['upscale_img_path'].head())  # Train 경로 확인
print("Validation paths:", val_df['upscale_img_path'].head())  # Validation 경로 확인
print("Test paths:", test_df['img_path'].head())  # Test 경로 확인


Train paths: 2251    /home/idp/lab/minjun/dl/project/upscale_train/...
482     /home/idp/lab/minjun/dl/project/upscale_train/...
5997    /home/idp/lab/minjun/dl/project/upscale_train/...
1436    /home/idp/lab/minjun/dl/project/upscale_train/...
2914    /home/idp/lab/minjun/dl/project/upscale_train/...
Name: upscale_img_path, dtype: object
Validation paths: 742      /home/idp/lab/minjun/dl/project/upscale_train/...
12485    /home/idp/lab/minjun/dl/project/upscale_train/...
6711     /home/idp/lab/minjun/dl/project/upscale_train/...
11125    /home/idp/lab/minjun/dl/project/upscale_train/...
10860    /home/idp/lab/minjun/dl/project/upscale_train/...
Name: upscale_img_path, dtype: object
Test paths: 0    /home/idp/lab/minjun/dl/project/test/TEST_0000...
1    /home/idp/lab/minjun/dl/project/test/TEST_0000...
2    /home/idp/lab/minjun/dl/project/test/TEST_0000...
3    /home/idp/lab/minjun/dl/project/test/TEST_0000...
4    /home/idp/lab/minjun/dl/project/test/TEST_0000...
Name: img_path, dtype

## Label-Encoding

In [16]:
le = preprocessing.LabelEncoder() # 라벨인코딩 /라벨(목표 변수)를 정수로 인코딩
# train, label의 라벨인코딩 과정 진행
train_df['label'] = le.fit_transform(train_df['label'])
val_df['label'] = le.transform(val_df['label'])

## CustomDataset

In [17]:
class CustomDataset(Dataset):
## 파일 경로와 라벨을 받아, 데이터를 로드하고 전처리하는 데이터셋 생성성
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        # 이미지 읽어오기
        image = cv2.imread(img_path)

        if self.transforms is not None:
            image = self.transforms(image = image)['image']

        # 라벨이 있다면 이미지와 함께 반환
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        # 라벨이 없다면 이미지만 반환환
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

# Compose는 여러 변환을 연속적으로적용할 수 있게 해주는 함수. (IMG 사이즈 224로 설정되어 있음)
# 이미지 크기조정, 정구화, 텐서로 변환 포함.
'''
Normalize(mean=0.485, 0.456, 0.406값은 각 채널별 평균)
std=(0.229, 0.224, 0.225 값은 각 채널별 표준편차)
max_pixel_value: 이미지의 최대 픽셀 값 (8비트의 경우 255가 최대값)
always_apply= Ture: 변환이 데이터셋의 모든 이미지에 대해 항상 적용.
p: 변환이 적용될 확률: (0~1 사이)
대부분의 경우 always_apply=True로 하고 p를 조절해서 사용
'''
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()])

# train 데이터셋 설정 및 DataLoader
train_dataset = CustomDataset(
    img_path_list=train_df['upscale_img_path'].values,
    label_list=train_df['label'].values,
    transforms=train_transform
)

# RandomSampler를 사용하여 데이터 순서를 랜덤화
train_sampler = RandomSampler(train_dataset)

train_loader = DataLoader(
    train_dataset,
    batch_size=CFG['BATCH_SIZE'],
    sampler=train_sampler,  # RandomSampler로 설정
    num_workers=0
)

# val 데이터셋 설정 및 DataLoader
val_dataset = CustomDataset(
    img_path_list=val_df['upscale_img_path'].values,
    label_list=val_df['label'].values,
    transforms=test_transform
)

# RandomSampler를 사용하여 데이터 순서를 랜덤화 (val에서도 shuffle 효과 적용 가능)
val_sampler = RandomSampler(val_dataset)

val_loader = DataLoader(
    val_dataset,
    batch_size=CFG['BATCH_SIZE'],
    sampler=val_sampler,  # RandomSampler로 설정
    num_workers=0
)

In [18]:
train_dataset.img_path_list

array(['/home/idp/lab/minjun/dl/project/upscale_train/TRAIN_02251.png',
       '/home/idp/lab/minjun/dl/project/upscale_train/TRAIN_00482.png',
       '/home/idp/lab/minjun/dl/project/upscale_train/TRAIN_05997.png',
       ...,
       '/home/idp/lab/minjun/dl/project/upscale_train/TRAIN_09444.png',
       '/home/idp/lab/minjun/dl/project/upscale_train/TRAIN_10564.png',
       '/home/idp/lab/minjun/dl/project/upscale_train/TRAIN_07952.png'],
      dtype=object)

## Model Define

In [19]:
import timm
print(timm.list_models("*deit3*"))


['deit3_base_patch16_224', 'deit3_base_patch16_384', 'deit3_huge_patch14_224', 'deit3_large_patch16_224', 'deit3_large_patch16_384', 'deit3_medium_patch16_224', 'deit3_small_patch16_224', 'deit3_small_patch16_384']


In [20]:
"""
class BaseModel(nn.Module):
    def __init__(self, model_name, num_classes, pretrained=True):
        super(BaseModel, self).__init__()
        # timm 모델을 로드하여 backbone으로 설정
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0  # 분류기를 제외한 backbone만 가져오기
        )
        # Backbone의 출력 크기를 확인하여 classifier 정의
        in_features = self.backbone.num_features
        self.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        # Backbone을 통해 특징 추출
        x = self.backbone(x)
        # Classifier로 최종 출력 생성
        x = self.classifier(x)
        return x
    
# 모델 이름과 클래스 수 설정
model_name = CFG['MODEL_NAME']
num_classes = 25

# 모델 생성
model = BaseModel(model_name, num_classes)

# 모델 구조 확인
print(model)
"""

"\nclass BaseModel(nn.Module):\n    def __init__(self, model_name, num_classes, pretrained=True):\n        super(BaseModel, self).__init__()\n        # timm 모델을 로드하여 backbone으로 설정\n        self.backbone = timm.create_model(\n            model_name,\n            pretrained=pretrained,\n            num_classes=0  # 분류기를 제외한 backbone만 가져오기\n        )\n        # Backbone의 출력 크기를 확인하여 classifier 정의\n        in_features = self.backbone.num_features\n        self.classifier = nn.Linear(in_features, num_classes)\n\n    def forward(self, x):\n        # Backbone을 통해 특징 추출\n        x = self.backbone(x)\n        # Classifier로 최종 출력 생성\n        x = self.classifier(x)\n        return x\n    \n# 모델 이름과 클래스 수 설정\nmodel_name = CFG['MODEL_NAME']\nnum_classes = 25\n\n# 모델 생성\nmodel = BaseModel(model_name, num_classes)\n\n# 모델 구조 확인\nprint(model)\n"

In [21]:
import torch
import torch.nn as nn
import timm

class EnsembleModel(nn.Module):
    def __init__(self, model_names, num_classes, pretrained=True):
        super(EnsembleModel, self).__init__()
        
        # 첫 번째 모델 로드
        self.model1 = timm.create_model(
            model_names[0],
            pretrained=pretrained,
            num_classes=0  # 분류기를 제외한 backbone만 가져오기
        )
        # 두 번째 모델 로드
        self.model2 = timm.create_model(
            model_names[1],
            pretrained=pretrained,
            num_classes=0  # 분류기를 제외한 backbone만 가져오기
        )
        
        # 두 모델의 출력 특징 크기 확인
        in_features_model1 = self.model1.num_features
        in_features_model2 = self.model2.num_features
        
        # 두 모델의 출력 크기를 합친 뒤 최종 분류기로 전달
        self.classifier = nn.Linear(in_features_model1 + in_features_model2, num_classes)

    def forward(self, x):
        # 각 모델의 특징 추출
        x1 = self.model1(x)
        x2 = self.model2(x)
        
        # 두 특징을 합침 (concat)
        x_concat = torch.cat((x1, x2), dim=1)
        
        # 최종 분류기
        out = self.classifier(x_concat)
        return out




# 모델 구조 확인
print(model)


EnsembleModel(
  (model1): ConvNeXt(
    (stem): Sequential(
      (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
    )
    (stages): Sequential(
      (0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
            (norm): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
            (mlp): GlobalResponseNormMlp(
              (fc1): Linear(in_features=192, out_features=768, bias=True)
              (act): GELU()
              (drop1): Dropout(p=0.0, inplace=False)
              (grn): GlobalResponseNorm()
              (fc2): Linear(in_features=768, out_features=192, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (shortcut): Identity()
            (drop_path): Identity()
          )
          (1): ConvN

In [22]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)  # 모델을 해당 디바이스로 옮김(cpu, gpu)
    # CrossEntropyLoss에 label_smoothing 적용
    criterion = nn.CrossEntropyLoss(label_smoothing=CFG['LABEL_SMOOTHING']).to(device)

    # 성능 기록 초기화
    best_score = 0
    best_model = None

    # 설정한 하이퍼파라미터의 epochs만큼 반복
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()  # 모델을 훈련모드로 설정
        train_loss = []
        train_correct = 0  # 올바른 예측 개수
        total_train = 0    # 총 데이터 개수

        # 반복을 통해서 배치 단위로 이미지와 라벨을 가져옴
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)  # 이미지를 실수형으로 변경한 후 device로 올림
            labels = labels.long().to(device)  # 데이터 타입 long으로 변경한 후 device로 올림 (int로 변경 시 오류 발생 가능)

            optimizer.zero_grad()  # 이전 그레디언트가 누적될 가능성이 있으니 초기화

            output = model(imgs)  # 모델의 이미지를 입력하여 출력을 얻음
            loss = criterion(output, labels)  # 손실 함수를 통해 손실 값을 계산함

            loss.backward()  # 손실에 대한 그레디언트 계산
            optimizer.step()  # 옵티마이저를 통해 모델의 가중치 업데이트

            train_loss.append(loss.item())  # 현재 배치에 대한 손실 값을 저장

            # 정확도 계산
            preds = output.argmax(1)  # 예측값
            train_correct += (preds == labels).sum().item()  # 올바른 예측 개수
            total_train += labels.size(0)  # 총 데이터 개수 증가

        # train 정확도 계산
        train_accuracy = train_correct / total_train

        # 검증 세트에서 모델의 성능을 평가
        _val_loss, _val_score, val_accuracy = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)  # 각 배치에서 계산된 모든 손실 값의 평균을 구함

        # 결과 출력
        print(f'Epoch [{epoch}], Train Loss: [{_train_loss:.5f}], Train Acc: [{train_accuracy:.5f}], '
              f'Val Loss: [{_val_loss:.5f}], Val Acc: [{val_accuracy:.5f}], Val F1 Score: [{_val_score:.5f}]')

        # scheduler이 설정되어 있다면 검증 성능에 따라 학습률을 조정
        if scheduler is not None:
            scheduler.step(_val_score)

        # 가장 좋은 성능을 보인 모델을 저장
        if best_score < _val_score:
            best_score = _val_score
            best_model = model

    return best_model


def validation(model, criterion, val_loader, device):
    model.eval()  # 평가모드
    val_loss = []
    preds, true_labels = [], []

    correct = 0  # 올바른 예측 개수
    total = 0    # 총 데이터 개수

    # 평가모드에서는 gradient 계산을 하지 않음
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.long().to(device)

            pred = model(imgs)
            loss = criterion(pred, labels)

            # 예측값 저장
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()

            # 정확도 계산
            correct += (pred.argmax(1) == labels).sum().item()
            total += labels.size(0)

            val_loss.append(loss.item())

        # 평균 손실 및 F1 점수 계산
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='macro')

        # 검증 정확도 계산
        val_accuracy = correct / total

    return _val_loss, _val_score, val_accuracy


In [26]:
# 모델 이름과 클래스 수 설정
model_names = [
    "convnextv2_large.fcmae_ft_in22k_in1k_384",
    "timm/deit3_large_patch16_224.fb_in22k_ft_in1k"
]
num_classes = 25

# 앙상블 모델 생성
model = EnsembleModel(model_names, num_classes) # 모델은 basemodel 가져옴
model.eval() #평가모드로 전환 (훈련모드가 아닌 평가모드를 불러온 이유가 뭐지?..)
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"]) # optimizer 'adam'으로 설정 / 학습률 위의 하이퍼파라미터

#학습률을 동적으로 조정하는 스케줄러 설정. 검증 성능이 개선되지 않으면 학습률 감소.
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.54 GiB of which 16.12 MiB is free. Including non-PyTorch memory, this process has 23.50 GiB memory in use. Of the allocated memory 22.86 GiB is allocated by PyTorch, and 206.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [18]:
test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad(): # gradient 초기화 없이 평가 진행
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)

            pred = model(imgs)

            preds += pred.argmax(1).detach().cpu().numpy().tolist()

    preds = le.inverse_transform(preds)
    return preds

preds = inference(infer_model, test_loader, device)

100%|██████████| 213/213 [00:03<00:00, 54.78it/s]


In [20]:
submit = pd.read_csv('/home/idp/lab/minjun/dl/project/sample_submission.csv')
submit['label'] = preds
submit.to_csv('/home/idp/lab/minjun/sub/baseline_submit.csv', index=False)